# MT Mount Characterization

From Craig Lage's notebook collection, edited to look at the TMA offset tests on the night of 3/9/2023

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import pickle as pkl
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient
from lsst.sitcom.vandv.efd import create_efd_client

In [ ]:
client = create_efd_client()

In [ ]:

async def az_el(start, end):
    ''' 
    Loads in data from mount encoders for a given time range
    Returns azimuth and elevation encoder positions, times when the mount is tracking the target, and 
    information about whether the azimuuth and elevation is in position
    '''
    az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['actualPosition', 'timestamp'],  start, end)
    el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['actualPosition', 'timestamp'],  start, end)    

    az_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['azimuth', 'taiTime'],  start, end)
    el_track = await client.select_time_series('lsst.sal.MTMount.command_trackTarget', \
                                            ['elevation', 'taiTime'],  start, end)  
    
    azPos = await client.select_time_series('lsst.sal.MTMount.logevent_azimuthInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
    azPos = azPos[azPos['inPosition']] # Select only the True values
    elPos = await client.select_time_series('lsst.sal.MTMount.logevent_elevationInPosition', \
                                            ['inPosition', 'private_kafkaStamp'],  start, end)
    elPos = elPos[elPos['inPosition']] # Select only the True values
    # print(len(azPos), len(elPos))
    
    slew_starts = await client.select_time_series('lsst.sal.MTPtg.command_raDecTarget', \
                                            ['ra', 'declination', 'private_sndStamp'],  start, end)
    
    # print(slew_starts)
    
    return az, el, az_track, el_track, azPos, elPos, slew_starts


def slew_settle_verification(az, el, az_track, el_track, azPos, elPos, slew_starts):
    ''' checking slew and settle for the 3.5 arcsec shifts and also more generally for larger shifts '''
    
    # plot azimuth and elevation
    # print(len(az_track), len(el_track))
    plt.figure()
    az['actualPosition'].plot()
    plt.ylabel('Azimuth (Degrees)')
    plt.show()
    plt.figure()
    el['actualPosition'].plot()
    plt.ylabel('Elevation (Degrees)')
    plt.show()
    
    # find out when the slew is occurring
    azs = az_track.values[:,0]
    els = el_track.values[:,0]
    
    
    times = az_track.values[:,1]
    start_slew_times_1 = []
    slew_dist_1 = []
    slew_times_1 = []
        
    # print(az_track)
    for i in range(1,len(az_track)):
        az_shift = abs(azs[i] - azs[i-1])
        el_shift = abs(els[i] - els[i-1])
        if (az_shift > 0.1) or (el_shift > 0.1):
            start_slew_times_1.append(times[i])
            az_shift_mod = az_shift * np.cos(els[i]*np.pi/180.0)
            shift = np.sqrt(el_shift*el_shift + az_shift_mod*az_shift_mod)
            slew_dist_1.append(shift)
    # print(len(start_slew_times_1))

    # Now in position timestamps

    inPos_1 = []
    azPosValues = azPos.values[:,1]
    elPosValues = elPos.values[:,1]

    for i in range(len(azPos)):
        if azPosValues[i] > elPosValues[i]:
            inPos_1.append(azPosValues[i])
        else:
            inPos_1.append(elPosValues[i])

    # print(len(inPos_1))

    # Now pair them up

    pairMin = 1.0
    pairMax = 10.0

    start_slew_times = []
    slew_dist = []
    slew_times = []
    inPos = []

    for i in range(len(start_slew_times_1)):
        for j in range(len(inPos_1)):
            deltaT = inPos_1[j] - start_slew_times_1[i] 
            if deltaT > pairMin and deltaT < pairMax:
                inPos.append(inPos_1[j])
                start_slew_times.append(start_slew_times_1[i])
                slew_times.append(deltaT)
                slew_dist.append(slew_dist_1[i])
        
    print(len(inPos), len(start_slew_times), len(slew_times), len(slew_dist))
    
    slew_dist = np.array(slew_dist)
    slew_times = np.array(slew_times)
    time_deltas = np.zeros(len(start_slew_times))
    
    for i, start_slew_time in enumerate(start_slew_times):
        k = np.argmin(np.abs((Time(start_slew_time, format='unix_tai', scale='utc') - \
                              Time(slew_starts.private_sndStamp, format='unix_tai', scale='utc')).to_value('s')))
        time_delta = (Time(start_slew_time, format='unix_tai', scale='utc') - Time(slew_starts.private_sndStamp[k], format='unix_tai', scale='utc'))
        time_deltas[i] = time_delta.to_value('s')
    
    slew_times = slew_times + time_deltas
    
    plt.hist(time_deltas)
    plt.xlabel('Time between MTPtg command send and slew start (s)')
    plt.show()
    
    # print(start_slew_times)
    # print(slew_starts)
    
    plt.subplots_adjust(wspace=0.5)
    plt.subplot(1,2,1)
    plt.hist(slew_times[(3 < slew_dist) & (slew_dist < 4)])
    plt.xlabel("Slew and settle time (seconds)")
    # plt.xlim(0.0, 10.0)
    plt.subplot(1,2,2)
    plt.scatter(slew_dist[(3 < slew_dist) & (slew_dist < 4)], slew_times[(3 < slew_dist) & (slew_dist < 4)])
    plt.ylabel("Slew and settle time (sec)")
    plt.xlabel("Slew distance (degrees)")
    plt.show()
    
    plt.subplots_adjust(wspace=0.5)
    plt.subplot(1,2,1)
    plt.hist(slew_times)
    plt.xlabel("Slew and settle time (seconds)")
    # plt.xlim(0.0, 10.0)
    plt.subplot(1,2,2)
    plt.scatter(slew_dist, slew_times)
    plt.ylabel("Slew and settle time(sec)")
    plt.xlabel("Slew distance (degrees)")
    plt.show()
    
    delta1 = 0
    delta = 240
    azPos_values = azPos.values[:,1]
    plt.subplot(1,1,1)
    plt.title("Azimuth Slew and Tracking")
    ax1 = az['actualPosition'].plot(color='red')
    for start_slew_time in start_slew_times:
        ss_time = Time(start_slew_time, format='unix_tai', scale='utc').isot  
        ax1.axvline(ss_time, color="black", linestyle="--")
    for inP in inPos:
        ip_time = Time(inP, format='unix_tai', scale='utc').isot  
        ax1.axvline(ip_time, color="blue", linestyle="--")
    for i in range(len(MTPtg.index)):
        start_time = Time(MTPtg.index[i], scale='utc').isot  
        ax1.axvline(start_time, color="red", linestyle="--")
    ax1.set_xlim((start+TimeDelta(delta1, format='sec')).isot, (start+TimeDelta(delta, format='sec')).isot)
    ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
    ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
    # ax1.set_ylim(-95, -75)
    ax1.legend()
    plt.show()
    return start_slew_times, slew_times, inPos
    
def jitter_verification(az, el, start_slew_times, slew_times, inPos):
    ''' checking jitter for the images '''
    azRmsVals = []
    elRmsVals = []
    imRmsVals = []
    
    for index in range(len(inPos) - 1):
        fig = plt.figure(figsize = (8,8))
        end_time = Time(inPos[index], format='unix_tai', scale='utc') + timedelta(seconds = 32)
        
        print(inPos[index])
        print(end_time)
        end_time = end_time.utc
        print(end_time.to_value('unix_tai'))
        
        plotAz = az[(az['timestamp'] > inPos[index]) & (az['timestamp'] < end_time.to_value('unix_tai'))]
        plotEl = el[(el['timestamp'] > inPos[index]) & (el['timestamp'] < end_time.to_value('unix_tai'))]
        
        ss_time = end_time.isot
        ip_time = Time(inPos[index], format='unix_tai', scale='utc').isot


        # Calculate the tracking errors
        az_vals = np.array(plotAz.values[:,0])
        el_vals = np.array(plotEl.values[:,0])
        times_az = plotAz.values[:,1]
        times_el = plotEl.values[:,1]
        # The fits are much better if the time variable
        # is centered in the interval
        time_delta_az = times_az[int(len(plotAz.values) / 2)]
        time_delta_el = times_el[int(len(plotEl.values) / 2)]
        fit_times_az = [(times_az[i]-time_delta_az) for i in range(len(times_az))]
        fit_times_el = [(times_el[i]-time_delta_el) for i in range(len(times_el))]

        # Fit with a polynomial
        az_fit = np.polyfit(fit_times_az, az_vals, 4)
        el_fit = np.polyfit(fit_times_el, el_vals, 4)
        az_model = np.polyval(az_fit, fit_times_az)
        el_model = np.polyval(el_fit, fit_times_el)

        # Errors in arcseconds
        az_error = (az_vals - az_model) * 3600
        el_error = (el_vals - el_model) * 3600
    
        # Drive velocities
        az_vel = (az_model[-1] - az_model[0]) / (fit_times_az[-1] - fit_times_az[0]) * 3600.0
        el_vel = (el_model[-1] - el_model[0]) / (fit_times_el[-1] - fit_times_el[0]) * 3600.0
    
        # Calculate RMS
        az_rms = np.sqrt(np.mean(az_error[np.abs(az_error) < 10 * np.std(az_error)] * az_error[np.abs(az_error) < 10 * np.std(az_error)]))
        el_rms = np.sqrt(np.mean(el_error[np.abs(el_error) < 10 * np.std(el_error)] * el_error[np.abs(el_error) < 10 * np.std(el_error)]))
        azRmsVals.append(az_rms)
        elRmsVals.append(el_rms)
        # Calculate Image impact RMS
        # We are less sensitive to Az errors near the zenith
        image_az_rms = az_rms * np.cos(el_vals[0] * np.pi / 180.0)
        image_el_rms = el_rms
        imRmsVals.append(np.sqrt(image_az_rms*image_az_rms + image_el_rms*image_el_rms))

        # fig.clear()
        plt.subplots_adjust(wspace=0.3, hspace=0.5)
        plt.suptitle(f"MT Mount Jitter - {ip_time}", fontsize = 18)
        plt.subplot(2,2,1)
        ax1 = plotAz['actualPosition'].plot(legend=True, color='red')
        ax1.axvline(ss_time, color="black", linestyle="--", label="Start slew")
        ax1.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
        ax1.set_title(f"Azimuth\nAve velocity={az_vel:.1f} arcsec/sec")
        ax1.set_ylabel("Degrees")
        ax1.legend()
        plt.subplot(2,2,2)
        ax3 = plotEl['actualPosition'].plot(legend=True, color='green')
        ax3.axvline(ss_time, color="black", linestyle="--", label="Start slew")
        ax3.axvline(ip_time, color="blue", linestyle="--", label="InPosition")
        ax3.set_title(f"Elevation\nAve velocity={el_vel:.1f} arcsec/sec")
        ax3.set_ylabel("Degrees")
        ax3.legend()
        plt.subplot(2,2,3)
        plt.plot(fit_times_az, az_error, color='red')
        plt.title(f"Azimuth RMS error = {az_rms:.3f} arcseconds\n"
                  f"  Image RMS error = {image_az_rms:.3f} arcseconds", fontsize=10)
        plt.ylim(-0.2,0.2)
        plt.xticks([])
        plt.ylabel("ArcSeconds")
        plt.subplot(2,2,4)
        plt.plot(fit_times_el, el_error, color='green')
        plt.title(f"Elevation RMS error = {el_rms:.3f} arcseconds\n"
                  f"  Image RMS error = {image_el_rms:.3f} arcseconds", fontsize=10)
        plt.ylim(-0.2,0.2)
        plt.xticks([])
        plt.ylabel("ArcSeconds")
        timestamp = ip_time.split('.')[0].replace('-','').replace(':','')
        plt.show()

    mount_data = {}
    mount_data['start_slew_times'] = start_slew_times
    mount_data['inPos'] = inPos
    mount_data['slew_times'] = slew_times
    # mount_data['slew_dist'] = slew_dist
    mount_data['azRmsVals'] = azRmsVals
    mount_data['elRmsVals'] = elRmsVals
    mount_data['imRmsVals'] = imRmsVals
    
    
    fig = plt.figure(figsize=(16,8))
    plt.suptitle("MT Mount RMS Jitter", fontsize = 18)
    azRmsVals = mount_data['azRmsVals']
    elRmsVals = mount_data['elRmsVals']
    imRmsVals = mount_data['imRmsVals']
    azMed = np.median(azRmsVals)
    elMed = np.median(elRmsVals)
    imMed = np.median(imRmsVals)
    plt.subplots_adjust(wspace=0.2)
    ax1 = plt.subplot(1,3,1)
    ax1.hist(azRmsVals)
    ax1.set_title(f"Azimuth RMS, N={len(azRmsVals)}")
    ax1.text(0.6,0.8, f"Median={azMed:.3f}", fontsize=12, transform=ax1.transAxes)
    # ax1.set_xlim(0,0.2)
    ax1.set_xlabel("RMS Jitter (arcseconds)")
    ax2 = plt.subplot(1,3,2)
    ax2.set_title(f"Elevation RMS, N={len(azRmsVals)}")
    ax2.hist(elRmsVals)
    ax2.text(0.5,0.8, f"Median={elMed:.3f}", fontsize=12, transform=ax2.transAxes)
    # ax2.set_xlim(0,0.2)
    ax2.set_xlabel("RMS Jitter (arcseconds)")
    ax3 = plt.subplot(1,3,3)
    ax3.set_title(f"Image Impact RMS, N={len(azRmsVals)}")
    ax3.hist(imRmsVals)
    ax3.text(0.6,0.8, f"Median={imMed:.3f}", fontsize=12, transform=ax3.transAxes)
    # ax3.set_xlim(0,0.2)
    ax3.set_xlabel("RMS Jitter (arcseconds)")
    plt.show()



In [ ]:
start = Time("2023-03-10T04:01:55", scale='utc')
end = Time("2023-03-10T04:09:58", scale='utc')
# first working 3.5 degree offset test
az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
start_slew_times, slew_times, inPos = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
jitter_verification(az, el, start_slew_times, slew_times, inPos)

In [ ]:
start = Time("2023-03-10T04:13:02", scale='utc')
end = Time("2023-03-10T04:45:04", scale='utc')
# first working 3.5 degree offset test
az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
start_slew_times, slew_times, inPos = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
jitter_verification(az, el, start_slew_times, slew_times, inPos)

In [ ]:
# random walk on 3/15
start = Time("2023-03-15T20:27:02", scale='utc')
end = Time("2023-03-15T21:10:04", scale='utc')
# first working 3.5 degree offset test
az, el, az_track, el_track, azPos, elPos, MTPtg = await az_el(start, end)
start_slew_times, slew_times, inPos = slew_settle_verification(az, el, az_track, el_track, azPos, elPos, MTPtg)
jitter_verification(az, el, start_slew_times, slew_times, inPos)